In [1]:
# Import modules
import pandas as pd
import json
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import preprocessor as p
import nltk
from nltk.tokenize import word_tokenize
import string
from collections import defaultdict
import contractions
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 500)


In [2]:
def clean_tweets_text(df, task_name):

    # Clean text with twitter-preprocessor
    clean = []
    for i, v in enumerate(df['text']):
        clean.append(p.clean(v))
    df["text"] = clean

    # Convert to lower case
    df["text"] = [entry.lower() for entry in df["text"]]

    # Expand Contractions


    def expand_contractions(s):
        expanded = []
        for word in s.split():
            expanded.append(contractions.fix(word))
        return(' '.join(expanded))


    df["text"] = [expand_contractions(entry) for entry in df['text']]

    # Remove punctuation marks
    df["text"] = [entry.translate(str.maketrans(
        '', '', string.punctuation)) for entry in df["text"]]

    # Tokenization
    # df['text']= [word_tokenize(entry) for entry in df['text']]
    df.to_csv(f'../res/preprocessed/{task_name}/{task_name}.csv', index=False)


### CLEAN task 1, 2 and 3 texts

In [6]:
task_df = pd.read_csv('../res/raw/subtask2.csv')
task_df.drop(["tweet_id",  "ID", "task2"], axis=1, inplace=True)

# Apply one hot encoding
task_df = task_df.join(pd.get_dummies(task_df['task1']))
task_df = task_df.dropna().drop(columns=['task1'])
task1_df = task_df
clean_tweets_text(task_df, 'task1')

In [8]:
task_df = pd.read_csv('../res/raw/subtask2.csv')
task_df.drop(["tweet_id",  "ID", "task1"], axis=1, inplace=True)

# Apply one hot encoding
task_df = task_df.join(pd.get_dummies(task_df['task2']))
task_df = task_df.dropna().drop(columns=['task2'])
task2_df = task_df
clean_tweets_text(task_df, 'task2')

In [5]:
df = pd.read_csv('../res/raw/raw_task3.csv')
df = df.rename(columns={'tweet_text':'text'})
clean_tweets_text(df, 'task3')

### Combine task1 and task2 one hot encoded

In [16]:
# df = pd.merge(task1_df, task2_df, left_index=True, right_index=True)
task2_df_temp = task2_df.drop(columns=['text'])
df = pd.concat([task1_df, task2_df_temp], axis=1)
df.to_csv('../res/preprocessed/task1_2/task1_2.csv')